In [1]:
import tester
import torch
torch.manual_seed(0)

In [2]:
tester.test_lr()

learning_rate test_loss test_accuracy
Patience triggered. End of learning
0.0001 0.6136885682741801 70.80555555555556
Patience triggered. End of learning
0.0001 0.6021314462025961 70.05555555555556
Patience triggered. End of learning
0.0001 0.6160043312443627 70.81111111111112
Patience triggered. End of learning
0.0001 0.5909867568148507 71.53888888888889
Patience triggered. End of learning
0.0001 0.5752439399560293 70.97777777777777
0.0005 0.5826872173282835 72.91111111111111
Patience triggered. End of learning
0.0005 0.6454056923588117 68.05555555555556
0.0005 0.6177597774399651 72.79444444444445
0.0005 0.559686122669114 70.5
0.0005 0.6658163857128885 72.29444444444445
Patience triggered. End of learning
0.001 0.5503532447748714 71.08888888888889
0.001 0.5332958350578944 73.72777777777777
0.001 0.5649370435211394 73.5
0.001 0.5444893257485496 73.34444444444445
0.001 0.5681578086482154 71.20555555555555
0.005 0.6629236828949716 60.294444444444444
0.005 0.5911501182450188 69.5166666666

In [3]:
tester.test_batchsizes()

batchsize test_loss test_accuracy
8 0.555255169669787 73.22222222222223
8 0.5445501108964285 72.61111111111111
8 0.5791431466738383 73.20555555555555
8 0.5606594979763031 73.64444444444445
8 0.5546131696965959 70.93888888888888
16 0.5424035588900248 74.32222222222222
16 0.5691758907503552 73.71666666666667
16 0.5845229708486133 74.20555555555555
16 0.58018848962254 72.45555555555555
Patience triggered. End of learning
16 0.6086435152424706 71.61111111111111
32 0.5685419349206818 72.73333333333333
32 0.5350181311368942 73.63888888888889
32 0.6001366260978911 72.01666666666667
32 0.5664363635910882 73.57222222222222
32 0.5964247783025106 72.45555555555555
64 0.5448077188597785 74.37777777777778
64 0.5268038792742623 73.90555555555555
64 0.5646111799610986 73.86111111111111
64 0.6702228900459077 69.96111111111111
Patience triggered. End of learning
64 0.6057475871509976 70.94444444444444
128 0.5378729187779956 73.55555555555556
128 0.5759552038378186 71.44444444444444
128 0.53636300563812

In [3]:
tester.test_dropout()

dropout_rate test_loss test_accuracy
Patience triggered. End of learning
0 0.5799761066834132 72.78333333333333
Patience triggered. End of learning
0 0.6190465316176414 69.59444444444445
0 0.7092984467744827 72.08888888888889
Patience triggered. End of learning
0 0.6016952676905526 73.05555555555556
Patience triggered. End of learning
0 0.5674670222732756 72.80555555555556
0.2 0.5549713886446423 73.53333333333333
0.2 0.5698155595196618 73.49444444444444
Patience triggered. End of learning
0.2 0.5573846383227242 72.16111111111111
0.2 0.547711095876164 74.19444444444444
Patience triggered. End of learning
0.2 0.5620317856470743 71.91666666666667
0.4 0.5226812975274192 73.00555555555556
0.4 0.5517480919758478 72.4888888888889
0.4 0.5246108455790414 73.27222222222223
0.4 0.538350342048539 72.98333333333333
0.4 0.5486693580945333 72.90555555555555
Patience triggered. End of learning
0.6 0.5834124618106418 69.05
0.6 0.537716183397505 72.09444444444445
0.6 0.551543578505516 72.05555555555556


In [4]:
tester.test_batchnorm()

batch_norm_used test_loss test_accuracy
True 0.6582122113969591 72.96111111111111
True 0.6546537321474817 70.15
True 0.536191561155849 72.88888888888889
True 0.5660892360740237 73.56666666666666
Patience triggered. End of learning
True 0.5475249571932687 72.33888888888889
Patience triggered. End of learning
False 0.5811678055259917 71.22777777777777
Patience triggered. End of learning
False 0.6219566315412521 71.12777777777778
Patience triggered. End of learning
False 0.5865743292702569 69.67222222222222
Patience triggered. End of learning
False 0.6034078399340311 70.07222222222222
Patience triggered. End of learning
False 0.5963921828402413 70.23333333333333


In [2]:
aggregated = tester.aggregate_csv_files()

In [3]:
aggregated[0]

,test_loss_mean,test_loss_std,test_accuracy_mean,test_accuracy_std
batchsize,,,,
8,0.559,0.013,72.724,1.064
16,0.577,0.024,73.262,1.183
32,0.573,0.026,72.883,0.707
64,0.582,0.057,72.610,2.010
128,0.570,0.053,71.881,1.834


In [5]:
aggregated[1]

,test_loss_mean,test_loss_std,test_accuracy_mean,test_accuracy_std
batch_norm_used,,,,
False,0.598,0.016,70.467,0.681
True,0.593,0.059,72.381,1.321


In [7]:
aggregated[2]

,test_loss_mean,test_loss_std,test_accuracy_mean,test_accuracy_std
dropout_rate,,,,
0.0,0.615,0.056,72.066,1.427
0.2,0.558,0.008,73.060,0.977
0.4,0.537,0.013,72.931,0.283
0.6,0.552,0.018,71.551,1.407
0.8,0.617,0.013,65.557,2.302


In [8]:
aggregated[3]

,test_loss_mean,test_loss_std,test_accuracy_mean,test_accuracy_std
learning_rate,,,,
0.0001,0.600,0.017,70.838,0.530
0.0005,0.614,0.044,71.311,2.061
0.0010,0.552,0.014,72.573,1.310
0.0050,0.605,0.035,67.646,4.164
0.0100,0.692,0.003,50.890,1.981
